In [2]:
import numpy as np
import cvxpy as opt
import pandas as pd
import csv
import pprint
import glob

In [3]:
"""
This cell will develop the time constraint matrix. We define this matrix as a 168 * 5 (i, j) 
matrix where i represent the 5 minute block of time and 
In this 1 and 0 matrix, 1 represents a scheduled class and 0 implies an empty spot. 
We can set a linear constraint in our linear program that says all must be less than 2. This 
constraint will be later in our optimization. 
"""
time_matrix = np.zeros((168,5))
time_constraint = (time_matrix <= 1)



In [4]:
"""
In this cell we will import all of the csv files and turn them into a parsable dataframe.
We will need to iterate through all of the files in data subfolder. 
"""
# Need to make a list of all file names
course_files = glob.glob('data/*.csv')

def pull_names(course_files):
    """
    Takes in a set of course files that have format {'data/*.csv'} and outputs a set of just department names
    """
    names = [] 
    for file in course_files:
        #take out the data
        pre, rest = file.split("/")
        #take out CSV
        index_name, blah = rest.split('.')
        #take out quarter
        department, quarter = index_name.split("-")
        names.append(department)
    return names

course_file_names = pull_names(course_files)
print("The following datasets have been properly uploaded " + str(course_file_names))

#Now we need to turn these bad boys into data frames. 
def create_dataframes(names_list, quarter):
    """
    Takes a set of CSV names (as a list) and outputs a dictionary of pandas frames 
    that can be easily accessed by department. 
    """
    pandas_frames = {}
    for file_name in names_list:
        pandas_frames[file_name] = pd.read_csv("data/" + file_name + "-" + quarter + ".csv")
        pandas_frames[file_name] = pandas_frames[file_name].reindex(index=pandas_frames[file_name].index[::-1]).reset_index()
    return pandas_frames

quarter = "WINTER2019"
frames = create_dataframes(course_file_names, quarter)

The following datasets have been properly uploaded ['MATH', 'PSTAT', 'CMPSC']


In [8]:
"""
Now need to convert to quantify and vectorize for formulating optimization problem. 
"""

def dataframe_cleaner_vectorizer(df, department):
    """
    Takes in a plaintext csv as imported from UCSB website and converts to columns that we can vectorize 
    easily.
    COLUMNS ARE AS FOLLOW:
    
    
    """
    new_df = pd.DataFrame()
    def time_str_to_blocks(time_string):
        """
        INPUT: Time String from CSV
        OUTPUT: 2 numbers representing start and stop written as five minute block (0-168) for our matrix
        takes the current time string that we have and converts into two separate entries with start
        start_time and end_end
        """
        start_str, end_str = time_string.split(" - ")
        start_dt, end_dt = pd.to_datetime(start_str), pd.to_datetime(end_str)
        def dt_to_fiver(dt): 
            """
            Take a datetime object and convert to our matrix notation
            """
            hour = (dt.hour - 8) * 6
            block = dt.minute/5
            return (hour+block)
        start_vec, end_vec = dt_to_fiver(start_dt), dt_to_fiver(end_dt)
        return start_vec, end_vec
    new_df["tups"] = df["Time"].apply(lambda string: time_str_to_blocks(string))
    new_df["start"] = new_df["tups"].apply(lambda x: x[0])
    new_df["end"] = new_df["tups"].apply(lambda x: x[1])
    new_df = new_df.drop(columns = "tups")
    new_df["dep"] = department
    new_df["cnum"] = df["Code"].apply(lambda x: x.split(" ")[-1])
    def day_str_to_np_array(string):
        """
        INPUT: Takes a string that has the {M T W R F} format 
        converts into a 5-D row vector of 1s and 0s
        OUTPUT: Numpy vector of 1 for days where the class is there.
        """
        array = np.zeros(5)
        if "M" in string:
            array[0] = 1
        if "T" in string:
            array[1] = 1
        if "W" in string:
            array[2] = 1
        if "R" in string:
            array[3] = 1
        if "F" in string:
            array[4] = 1
        return array
    new_df["dayarr"] = df["Days"].apply(lambda daystr: day_str_to_np_array(daystr))
    return new_df

In [9]:
dataframe_cleaner_vectorizer(frames["MATH"], "MATH").head()

,start,end,dep,cnum,dayarr
0,12.0,22.0,MATH,137A,"[1.0, 0.0, 1.0, 0.0, 1.0]"
1,36.0,45.0,MATH,124A,"[0.0, 1.0, 0.0, 1.0, 0.0]"
2,30.0,39.0,MATH,122A,"[0.0, 1.0, 0.0, 1.0, 0.0]"
3,18.0,27.0,MATH,122A,"[0.0, 1.0, 0.0, 1.0, 0.0]"
4,24.0,33.0,MATH,120,"[1.0, 0.0, 1.0, 0.0, 0.0]"


In [7]:
#Optimization Solver Cell
cons = [time_constraint]
